In [1]:

using ITensors
using Pkg
using NPZ

In [2]:
N = 64
# Make an array of N Index objects with alternating
# "S=1/2" and "S=1" tags on odd versus even sites
# (The first argument n->isodd(n) ... is an 
# on-the-fly function mapping integers to strings)

sites = siteinds("S=1/2", N)
entangle_E = []
pa = []
P0_list = []
P1_list = []

angle_list = [0.0, 0.05*pi, 0.1*pi, 0.15*pi, 0.2*pi, 0.25*pi, 0.3*pi, 0.35*pi, 0.4*pi, 0.45*pi, 0.5*pi]
ITensors.op(::OpName"P0",::SiteType"S=1/2") =
 [1 0
  0 0]
ITensors.op(::OpName"P1",::SiteType"S=1/2") =
 [0 0
  0 1]

for angle in angle_list
    all_tensors = []
    os = OpSum()
    os -= 4.0*cos(angle)^2, "Sx",1,"Sz",2
    os -= 4.0*cos(angle)*sin(angle), "Sz",1,"Sz",2
    os += 4.0*sin(angle)^2, "Sz",1,"Sx",2
    os += 4.0*cos(angle)*sin(angle), "Sx",1,"Sx",2
    
    os -= 4.0*cos(angle)^2, "Sz", N-1,"Sx",N
    os -= 4.0*cos(angle)*sin(angle), "Sz", N-1,"Sz",N
    os += 4.0*sin(angle)^2, "Sx", N-1,"Sz",N
    os += 4.0*cos(angle)*sin(angle), "Sx", N-1,"Sx",N
    
    for j=1:N-2
        os -= 8.0*cos(angle)^3, "Sz",j,"Sx",j+1,"Sz",j+2
        os += 8.0*cos(angle)^2*sin(angle), "Sx",j,"Sx",j+1, "Sz", j+2
        os -= 8.0*cos(angle)^2*sin(angle), "Sz",j,"Sz",j+1, "Sz", j+2
        os += 8.0*cos(angle)^2*sin(angle), "Sz",j,"Sx",j+1,"Sx", j+2
        os += 8.0*cos(angle)*sin(angle)^2, "Sx",j,"Sz",j+1, "Sz", j+2
        os += 8.0*cos(angle)*sin(angle)^2, "Sz",j,"Sz",j+1, "Sx", j+2
        os -= 8.0*cos(angle)*sin(angle)^2, "Sx",j,"Sx",j+1,"Sx", j+2
        os -= 8.0*sin(angle)^3, "Sx",j,"Sz",j+1,"Sx", j+2
    end
    
    H = MPO(os, sites)
    
    nsweeps = 5
    maxdim = [10,20,40,80,200]
    mindim = [2, 2, 2, 2, 2]
    cutoff = [1E-6]
    noise = [1E-5, 1E-6, 1E-7, 1E-8, 0.0]
    psi0 = randomMPS(sites, 10)
    ang = round(angle, digits = 3)
    
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, mindim, cutoff, noise)
    save_dir = "mps_tensors"
    isdir(save_dir) || mkdir(save_dir)  # Create the directory if it doesn't exist
    
    # Save the initial tensor as an .npy file
    
    tensor = psi[1]
    tensor_array = Array(tensor, siteinds(psi, 1), linkinds(psi, 1))  # Convert the ITensor to a dense array
    file_path = joinpath(save_dir, "cluster_tensor_init_$(N)_angle_$(ang).npy")  # File path for the .npy file
    npzwrite(file_path, tensor_array)  # Save the array as an .npy file
    # Iterate over each site in the MPS and save the tensor as an .npy file
    
    for i in 2:length(psi)-1
        tensor = psi[i] # Convert the ITensor to a dense array
        tensor_array = Array(tensor, siteinds(psi, i), linkinds(psi, i), linkinds(psi, i-1))  # Convert the ITensor to a dense array
        push!(all_tensors, tensor_array) 
    end
    
    file_path = joinpath(save_dir, "cluster_tensor_$(N)_angle_$(ang).npy")  # File path for the .npy file
    stacked_tensor = cat(all_tensors..., dims=ndims(all_tensors[1]) + 1)
    npzwrite(file_path, stacked_tensor)  # Save the array as an .npy file
    tensor = psi[length(psi)]
    tensor_array = Array(tensor, siteinds(psi, length(psi)), linkinds(psi, length(psi)-1))  # Convert the ITensor to a dense array
    file_path = joinpath(save_dir, "cluster_tensor_last_$(N)_angle_$(ang).npy")  # File path for the .npy file
    npzwrite(file_path, tensor_array)  # Save the array as an .npy file
    
    sample_array = [] 
    for i in 1:1024
        push!(sample_array, sample(psi))
    end
    sample_array = transpose(hcat(sample_array...))
    file_path = joinpath(save_dir, "cluster_sample_$(N)_angle_$(ang).npy")  # File path for the .npy file
    npzwrite(file_path, Array(sample_array))  # Save the array as an .npy file
end


After sweep 1 energy=-63.99976677861847  maxlinkdim=3 maxerr=8.00E-04 time=26.444
After sweep 2 energy=-64.00000000000011  maxlinkdim=2 maxerr=8.59E-10 time=0.061
After sweep 3 energy=-64.00000000000006  maxlinkdim=2 maxerr=8.88E-16 time=0.073
After sweep 4 energy=-64.00000000000007  maxlinkdim=2 maxerr=6.66E-16 time=0.048
After sweep 5 energy=-64.00000000000014  maxlinkdim=2 maxerr=7.77E-16 time=0.301
After sweep 1 energy=-63.99957442244069  maxlinkdim=5 maxerr=1.15E-03 time=0.069
After sweep 2 energy=-63.99999999999982  maxlinkdim=2 maxerr=1.77E-09 time=0.059
After sweep 3 energy=-64.00000000000024  maxlinkdim=2 maxerr=6.66E-16 time=0.048
After sweep 4 energy=-64.00000000000003  maxlinkdim=2 maxerr=5.55E-16 time=0.048
After sweep 5 energy=-64.0  maxlinkdim=2 maxerr=5.76E-16 time=0.043
After sweep 1 energy=-63.999714780431134  maxlinkdim=4 maxerr=1.58E-03 time=0.060
After sweep 2 energy=-63.99999999999981  maxlinkdim=2 maxerr=1.40E-09 time=0.047
After sweep 3 energy=-64.00000000000001